# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

Import the open-source software libraries.

In [150]:
import numpy as np 
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks
Import the constituents of the S&P 500.

These constituents change over time, so in an ideal world we would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.
 
There's a static version of the S&P 500 constituents available here. [Click this link to download](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder.

Further, Import these stocks to Jupyter Notebook file.

In [151]:
stocks = pd.read_csv('sp_500_stocks.csv')
type(stocks)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Import IEX Cloud API token --- The data provider in use [ Create an account there to get the private key]

API token should be stored in a `secrets.py` file that doesn't get pushed to the local Git repository. 

** Important - This project is not using a sandbox API token, which means that the data we'll use is Real world data and not random.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download `secrets.py` file. Move the file into the same directory as this Jupyter Notebook.

In [152]:
from secretsZ import IEX_CLOUD_API_TOKEN

## Making First API Call

Structure the API calls to IEX cloud. 

The following information is required from the API:

* Market capitalization for each stock
* Price of each stock



In [153]:
symbol = 'AAPL'
# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
# data = requests.get(api_url).json()
# print(data['symbol'])
response = requests.get(api_url)
# print(response.status_code)
data = response.json()[0]
print(data)

{'avgTotalVolume': 67098513, 'calculationPrice': 'close', 'change': 1.27, 'changePercent': 0.0077, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 166.13, 'iexCloseTime': 1680551999900, 'iexLastUpdated': 1680551999900, 'iexMarketPercent': 30.617822587937425, 'iexOpen': 164.27, 'iexOpenTime': 1680528600089, 'iexRealtimePrice': 166.13, 'iexRealtimeSize': 100, 'iexVolume': 677205, 'lastTradeTime': 1680551999966, 'latestPrice': 166.17, 'latestSource': 'Close', 'latestTime': 'April 3, 2023', 'latestUpdate': 1680552000104, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 2629132766820, 'oddLotDelayedPrice': None, 'od

## Parsing the API Call

The API call that we executed in the last code block contains all of the information required to build the equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [154]:
price = data["latestPrice"]
# Real time data unlike sandbox
print(price)
market_cap = data["marketCap"]
print(market_cap/1000000000000) #trillions


166.17
2.62913276682


## Adding Our Stocks Data to a Pandas DataFrame

Adding our stock's price and market capitalization to a pandas DataFrame.

In [155]:
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
final_dataframe = pd.DataFrame(columns= my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [156]:
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)
# warnings because append() will soon be depreciated and replaced by concat, so to not display the warning, first two lines are there
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        "N/A"
    ],
        index = my_columns # should be within dataframe and outside the series
    ),
    
    ignore_index = True # mandatory for append()
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,166.17,2629132766820,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [157]:
final_dataframe = pd.DataFrame(columns= my_columns)
# for loop for tickers
for stock in stocks['Ticker'][:10]:
    #make api calls for each stock
    api_url = f'https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}'
    response = requests.get(api_url) # very very slow ; multiple api request
    data = response.json()[0] 
    final_dataframe =final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
            index = my_columns
        ),
            ignore_index = True
        )

In [158]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,137.86,40765457455,N/A
1,AAL,14.42,9413613800,N/A
2,AAP,122.69,7272300191,N/A
3,AAPL,166.17,2629132766820,N/A
4,ABBV,160.42,283003625471,N/A
5,ABC,162.28,32822458749,N/A
6,ABMD,381.02,17180642928,N/A
7,ABT,101.59,176557957810,N/A
8,ACN,285.84,189396348314,N/A
9,ADBE,380.08,174295460798,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - We'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [159]:
# IEX limits to 100 tickers per Batch call so break 505 into 100 limit --> split list into sublist i.e. chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [160]:
symbol_groups = list(chunks(stocks['Ticker'], 100)) # ticker is the header; if its just stocks --> it will give dataframe ; if stocks[ticker] it will return dataseries, which we can pass into chunks functions.
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns= my_columns)
#   use strings of symbol_strings to execute Batch API calls
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW' or symbol == 'DISCA':
            continue
        else:
            final_dataframe =  final_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        data[symbol]['quote']['latestPrice'], # multiple levels of parsing
                        data[symbol]['quote']['marketCap'],  # multiple levels of parsing
                        'N/A'
                    ],


                index = my_columns
                ),
                ignore_index=True

            )
final_dataframe


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,137.86,40765457455,N/A
1,AAL,14.42,9413613800,N/A
2,AAP,122.69,7272300191,N/A
3,AAPL,166.17,2629132766820,N/A
4,ABBV,160.42,283003625471,N/A
...,...,...,...,...
496,YUM,133.21,37313168430,N/A
497,ZBH,128.04,26757831978,N/A
498,ZBRA,311.65,16020287844,N/A
499,ZION,29.12,4312644569,N/A


## Calculating the Number of Shares to Buy



In [161]:
portfolio_size = input("Enter the value of your Portfolio: ")
try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("Thats not a number.\nPlease try again ")
    portfolio_size = input("Enter the value of your Portfolio: ")
    val = float(portfolio_size)

Enter the value of your Portfolio: 10000000
10000000.0


In [162]:
position_size = val/len(final_dataframe.index)
# Using position size calculate the number of shares of each stock to get that position size [Use floor value]
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/ final_dataframe.loc[i, 'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,137.86,40765457455,144
1,AAL,14.42,9413613800,1384
2,AAP,122.69,7272300191,162
3,AAPL,166.17,2629132766820,120
4,ABBV,160.42,283003625471,124
...,...,...,...,...
496,YUM,133.21,37313168430,149
497,ZBH,128.04,26757831978,155
498,ZBRA,311.65,16020287844,64
499,ZION,29.12,4312644569,685


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization.

### Initializing our XlsxWriter Object

In [163]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
# pass the pandas df into this object to specify which tab of the excel file it should be saved to
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. Four main formats for the Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [165]:
background_color = '#10565E'#'#0a0a23'
font_color = '#ffffff'
# create formats for referencing 
string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)


### Applying the Formats to the Columns of Our `.xlsx` File

Use `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of the spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [145]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()

# formatting the column title headers
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)



Let's simplify this by putting it in 2 loops:

In [166]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [167]:
writer.save()